# Notebook Imports

In [1]:
import os
import numpy as np
import tensorflow as tf

from time import strftime
from PIL import Image

# Constants

In [2]:
X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'

NR_CLASSES = 10
VALIDATION_SIZE = 10000

IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_HEIGHT*IMAGE_WIDTH*CHANNELS

# Load the Data

In [3]:
# Loading the data into an array
y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)

y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)

x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)

x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)

# Data Pre-Processing

In [4]:
# Rescale the features
x_train_all, x_test = x_train_all / 255.0 , x_test / 255.0

### i) Convert target values to one-hot encoding

In [5]:
y_train_all = np.eye(NR_CLASSES)[y_train_all]

y_test = np.eye(NR_CLASSES)[y_test]

### ii) Create Validation dataset from training

In [6]:
x_val = x_train_all[:VALIDATION_SIZE]

y_val = y_train_all[:VALIDATION_SIZE]

x_train = x_train_all[VALIDATION_SIZE:]

y_train = y_train_all[VALIDATION_SIZE:]

# Setup TensorFlow Graph

In [58]:
# Add/ Define tf placeholder(for features)
X = tf.placeholder(tf.float32, shape=[None, TOTAL_INPUTS])

# placeholder for labels
Y = tf.placeholder(tf.float32, shape=[None, NR_CLASSES])

# Neural Network Architecture

### i) Hyper-Parameters

In [59]:
nr_epochs = 50
learning_rate = 1e-3          # 0.001

# 1st hidden layer(contains 512 neurons)
n_hidden1=512
n_hidden2=64

### ii) Create Layers

In [60]:
def setup_layer(input, weight_dim, bias_dim, name):
    
    with tf.name_scope(name):
        initial_w = tf.truncated_normal(shape=weight_dim, stddev=0.1, seed=42)
        w = tf.Variable(initial_value=initial_w)

        initial_b = tf.constant(value=0.0, shape=bias_dim)
        b = tf.Variable(initial_value=initial_b)

        layer_in = tf.matmul(input, w) + b
        
        if name=='out':
            layer_out = tf.nn.softmax(layer_in)
            
        else:
            layer_out = tf.nn.relu(layer_in)
            
            
        return layer_out


### iii) Setup Layers

In [61]:
layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1], bias_dim=[n_hidden1], name='Layer_1')

layer_2 = setup_layer(layer_1, weight_dim=[n_hidden1, n_hidden2], bias_dim=[n_hidden2], name='Layer_2')

output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES], bias_dim=[NR_CLASSES], name='out')

# Loss, Optimization

### i) Loss Funtion

In [62]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output))

### ii) Optimizer

In [63]:
optimizer = tf.train.AdamOptimizer(learning_rate)

train_step = optimizer.minimize(loss)

### iii) Accuracy Metric

In [64]:
correct_pred = tf.equal(tf.argmax(output, axis=1), tf.argmax(Y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Create and Run Session

In [65]:
sess = tf.Session()

### i) Initialize all Variables

In [66]:
init = tf.global_variables_initializer()

# feed the variables in the session
sess.run(init)

# Batch (Split) the Data

In [67]:
size_of_batch = 1000

num_examples = y_train.shape[0]
nr_iterations = int(num_examples / size_of_batch)

index_in_epoch = 0 

In [68]:
def next_batch(batch_size, data, labels):
    
    global num_examples
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > num_examples:
        start = 0
        index_in_epoch = batch_size
    
    end = index_in_epoch
    
    return data[start:end], labels[start:end]

# Training the Model

In [69]:
for epoch in range(nr_epochs):
    
    # ======================= Training Dataset ======================= #
    for i in range(nr_iterations):
        
        batch_x, batch_y = next_batch(batch_size=size_of_batch, data=x_train, labels=y_train)
        
        # Feed dictionary(to be fed into session for calculation)
        feed_dictionary = {X:batch_x, Y:batch_y}
        
        sess.run(train_step, feed_dict=feed_dictionary)
        
  
    batch_accuracy = sess.run(fetches=[accuracy], feed_dict=feed_dictionary)
    
    print(f'Epoch {epoch} \t | Training Accuracy = {batch_accuracy}')

print('Done Training!')

Epoch 0 	 | Training Accuracy = [0.859]
Epoch 1 	 | Training Accuracy = [0.867]
Epoch 2 	 | Training Accuracy = [0.871]
Epoch 3 	 | Training Accuracy = [0.875]
Epoch 4 	 | Training Accuracy = [0.875]
Epoch 5 	 | Training Accuracy = [0.965]
Epoch 6 	 | Training Accuracy = [0.984]
Epoch 7 	 | Training Accuracy = [0.986]
Epoch 8 	 | Training Accuracy = [0.984]
Epoch 9 	 | Training Accuracy = [0.986]
Epoch 10 	 | Training Accuracy = [0.988]
Epoch 11 	 | Training Accuracy = [0.988]
Epoch 12 	 | Training Accuracy = [0.989]
Epoch 13 	 | Training Accuracy = [0.989]
Epoch 14 	 | Training Accuracy = [0.989]
Epoch 15 	 | Training Accuracy = [0.99]
Epoch 16 	 | Training Accuracy = [0.991]
Epoch 17 	 | Training Accuracy = [0.991]
Epoch 18 	 | Training Accuracy = [0.991]
Epoch 19 	 | Training Accuracy = [0.991]
Epoch 20 	 | Training Accuracy = [0.991]
Epoch 21 	 | Training Accuracy = [0.991]
Epoch 22 	 | Training Accuracy = [0.991]
Epoch 23 	 | Training Accuracy = [0.993]
Epoch 24 	 | Training Accur

# Make a Prediction

### i) Provide Input

In [70]:
img = Image.open('MNIST/test_img.png')

### ii) Process the Input

In [71]:
# Convert the image into grey scale(black and white)
bw = img.convert('L')

# convert the image into an array
img_array = np.invert(bw)

# Flattening the arrray
test_img = img_array.ravel()

# feed it to tensorflow(using session(sess))
prediction = sess.run(fetches=tf.argmax(output, axis=1),feed_dict={X:[test_img]})

### iii) Output

In [72]:
print(f'Prediction for the test image is {prediction}')

Prediction for the test image is [2]


# Accuracy of the Model

In [73]:
# Accuracy over the test dataset
test_accuracy = sess.run(fetches=accuracy, feed_dict={X:x_test, Y:y_test})

print(f'Accuracy on test set is {test_accuracy:0.2%}')

Accuracy on test set is 97.94%


# Reset


In [76]:
sess.close()
tf.reset_default_graph()

# The End